<a href="https://colab.research.google.com/github/HinePo/PDF-Document-Indexing/blob/master/Elastic_Search_Document_Indexing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Document Indexing

Using python and Elastic Search.

# Install and import packages

In [ ]:
pip install elasticsearch

     |████████████████████████████████| 225kB 2.9MB/s 


In [ ]:
pip install PyPDF2

     |████████████████████████████████| 81kB 2.3MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-cp36-none-any.whl size=61086 sha256=141cb672d89800c82bd87be23349a332f9eac1759554dc897511c82d8e6bb3d7
  Stored in directory: /root/.cache/pip/wheels/53/84/19/35bc977c8bf5f0c23a8a011aa958acd4da4bbd7a229315c1b7
Successfully built PyPDF2


In [ ]:
from elasticsearch import Elasticsearch
import os
import glob
import PyPDF2
import pandas as pd

# Locate PDFs

In [ ]:
os.chdir("/content/temp pdfs")
files = glob.glob("*.*")

In [ ]:
# number of files in folder
len(files)

5

In [ ]:
for book in files:
  print(book)

Medeia - Eurípedes.pdf
Hamlet - Shakespeare - Original.pdf
O Mal-Estar na Civilização  - Sigmund Freud.pdf
Platao_A_Republica.pdf
O mito de Sísifo - Albert Camus.pdf


# Extract texts from PDFs

In [ ]:
# create a dataframe with extracted texts

def extractPDFfiles(files):

  this_loc = 1
  df = pd.DataFrame(columns = ("name", "content")) # create empty dataframe

  for file in files: # for each file/document in folder
    pdfFileObj = open(file, 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    n_pages = pdfReader.numPages
    this_doc = '' # create empty document object

    for i in range(n_pages): # for each page in file/document
      pageObj = pdfReader.getPage(i)
      this_text = pageObj.extractText()
      this_doc += this_text # concatenate texts from each page
    
    df.loc[this_loc] = file, this_doc # put extracted information in the dataframe
    this_loc = this_loc + 1 # go to the next row on the dataframe
  
  return df

In [ ]:
df = extractPDFfiles(files)

In [ ]:
df.head()

,name,content
1,Medeia - Eurípedes.pdf,EURÍPEDES MEDEIA Distribuído por...
2,Hamlet - Shakespeare - Original.pdf,"TheTragedyofHamlet,Princeof\nDenmark\nASCIItex..."
3,O Mal-Estar na Civilização - Sigmund Freud.pdf,Sigmund FreudO MAL ESTAR NA CIVILIZA•†OTexto c...
4,Platao_A_Republica.pdf,\n \n \n \n \n \n \n \n \n \n \nPla...
5,O mito de Sísifo - Albert Camus.pdf,O MITO E A REALIDADE \n Entre as maiores manif...


In [ ]:
df.iloc[3,1]

' \n \n  \n \n \n \n          \n \n \n \n \nPlatão\n \n \n \nA República\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nLIVRO I\n \n \n \n \n \n \n        SÓCRATES \nŠ\n Fui ontem ao Pireu com Glauco, filho de\n \nAríston, para orar à\n deusa,\n e também para me certificar de\n \ncomo seria a festividade, que eles promoviam pela primeira\n \nvez. A procissão dos atenienses foi bastante agradável, embora\n \nnão me parecesse superior à realizada pelos trácios. Após termos\n \norado e admirado a cerimônia, \nestávamos regressando à cidade\n \nquando, no caminho, fomos vistos a distância por Polemarco,\n \nfilho de Céfalo. Ele mandou seu jovem escravo correr até nós,\n \npara nos pedir que o esperássemos. O servo puxou\n-\nme pela\n \ncapa, por trás, dizendo:\n \n        \nŠ\n Polemarco p\nede que o esperem.\n \n        Virei\n-\nme e indaguei onde seu amo se encontrava.\n \n        \nŠ\n Está vindo atrás de mim \nŠ\n respondeu o jovem. \nŠ\n \nEsperem\n-\nno.\n \n        \nŠ\n Ev

In [ ]:
df.shape[0], df.shape[1] # files, columns

(5, 2)

# Indexing documents

In [ ]:
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.0.0-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.0.0-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.0.0
# start server
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.0.0/bin/elasticsearch'], 
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)  # as daemon
                 )

In [ ]:
# wait a bit then test
!curl -X GET "localhost:9200/"

{
  "name" : "e89b196502a9",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "yER5SBDNQwi31J2WVY3Svw",
  "version" : {
    "number" : "7.0.0",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "b7e28a7",
    "build_date" : "2019-04-05T22:55:32.697037Z",
    "build_snapshot" : false,
    "lucene_version" : "8.0.0",
    "minimum_wire_compatibility_version" : "6.7.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [ ]:
# instantiate
es = Elasticsearch()

In [ ]:
# test connection to the elastic search cluster
es.ping()

True

## Create Index

In [ ]:
# create index
es.indices.create(index='ex', ignore=[400, 404])

{'acknowledged': True, 'index': 'ex', 'shards_acknowledged': True}

In [ ]:
df.columns

Index(['name', 'content'], dtype='object')

In [ ]:
col_names = df.columns
for row_number in range(df.shape[0]): # for all rows in the dataframe
  body = dict([(x, str(df.iloc[row_number][x])) for x in col_names]) # create dictionary for each field
  es.index(index = 'ex', body = body) # indexing body on the index 'ex'

In [ ]:
body

{'content': 'O MITO E A REALIDADE \n Entre as maiores manifestações da consciência crítica neste século, \na presença de Camus é certamente uma das mais generosas. \n\nSobretudo agora, no final do milênio, quando tantas das suas \n\nreflexões podem ser redescobertas como advertências ou \n\nﬁdiagnósticosﬂ de espantosa acuidade e rigor intelectual. Não há \ncomo duvidar de que o homem dos nossos dias tem tudo para \nabrigar conflitos ainda mais intensos - e mais devastadores, ou mais \n\nfecundos - que os de todas as outras épocas. É certo que ele \n\ncontou com enormes precursores, mestres que foram ao fundo do \n\ndesenvolvimento moderno de suas emoções - e suas razões - como \nNietzsche, Dostoiévski, Proust, Kierkegaard, Kafka (para só \nficarmos em alguns dos nomes mais caros a Camus), e chega, hoje \n\nem dia, aos desdobramentos efetivos e consistentes das revoluções \n\nde Darwin, Marx, Freud, Einstein. Mas, até mesmo por isso tudo, \n\nﬁos homens presentesﬂ, n™ "a vida presenteﬂ,

# Search

In [ ]:
search_results = es.search(index = 'ex',
                           body = {"_source" : "name", # return name column
                                   "query" : {
                                       'match_phrase' : {"content" : "sombra"}, # word/phrase to search for
                                             }
                                   })

In [ ]:
search_results['hits']['total']

{'relation': 'eq', 'value': 9}

In [ ]:
search_results['hits']['total']['value']

9

In [ ]:
search_results['hits']

{'hits': [{'_id': 'LDRNGXQBbRjn8Q995xh1',
   '_index': 'ex',
   '_score': 1.4048824,
   '_source': {'name': 'Platao_A_Republica.pdf'},
   '_type': 'books'},
  {'_id': 'MTSYGXQBbRjn8Q99WRjZ',
   '_index': 'ex',
   '_score': 1.4048824,
   '_source': {'name': 'Platao_A_Republica.pdf'},
   '_type': 'books'},
  {'_id': 'NjTMGXQBbRjn8Q99wRj-',
   '_index': 'ex',
   '_score': 1.4048824,
   '_source': {'name': 'Platao_A_Republica.pdf'},
   '_type': 'books'},
  {'_id': 'OzTWGXQBbRjn8Q99uRiQ',
   '_index': 'ex',
   '_score': 1.4048824,
   '_source': {'name': 'Platao_A_Republica.pdf'},
   '_type': 'books'},
  {'_id': 'LTRNGXQBbRjn8Q996BhA',
   '_index': 'ex',
   '_score': 0.82817084,
   '_source': {'name': 'O mito de Sísifo - Albert Camus.pdf'},
   '_type': 'books'},
  {'_id': '1',
   '_index': 'ex',
   '_score': 0.82817084,
   '_source': {'name': 'O mito de Sísifo - Albert Camus.pdf'},
   '_type': 'books'},
  {'_id': 'MjSYGXQBbRjn8Q99WhhK',
   '_index': 'ex',
   '_score': 0.82817084,
   '_source

In [ ]:
search_results

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': 'LDRNGXQBbRjn8Q995xh1',
    '_index': 'ex',
    '_score': 1.4048824,
    '_source': {'name': 'Platao_A_Republica.pdf'},
    '_type': 'books'},
   {'_id': 'MTSYGXQBbRjn8Q99WRjZ',
    '_index': 'ex',
    '_score': 1.4048824,
    '_source': {'name': 'Platao_A_Republica.pdf'},
    '_type': 'books'},
   {'_id': 'NjTMGXQBbRjn8Q99wRj-',
    '_index': 'ex',
    '_score': 1.4048824,
    '_source': {'name': 'Platao_A_Republica.pdf'},
    '_type': 'books'},
   {'_id': 'OzTWGXQBbRjn8Q99uRiQ',
    '_index': 'ex',
    '_score': 1.4048824,
    '_source': {'name': 'Platao_A_Republica.pdf'},
    '_type': 'books'},
   {'_id': 'LTRNGXQBbRjn8Q996BhA',
    '_index': 'ex',
    '_score': 0.82817084,
    '_source': {'name': 'O mito de Sísifo - Albert Camus.pdf'},
    '_type': 'books'},
   {'_id': '1',
    '_index': 'ex',
    '_score': 0.82817084,
    '_source': {'name': 'O mito de Sísifo - Albert Camus.pdf'},
    

In [ ]:
res = es.search(index = 'ex',
                body = {"_source" : "name", # return name column
                        "query" : {
                            'match_phrase' : {"content" : "vinho"}, # word/phrase to search for
                                  }
                        })

print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print(hit["_source"]['name']) # print only the name

Got 9 Hits:
Platao_A_Republica.pdf
Platao_A_Republica.pdf
Platao_A_Republica.pdf
Platao_A_Republica.pdf
O mito de Sísifo - Albert Camus.pdf
O mito de Sísifo - Albert Camus.pdf
O mito de Sísifo - Albert Camus.pdf
O mito de Sísifo - Albert Camus.pdf
O mito de Sísifo - Albert Camus.pdf


In [ ]:
res = es.search(index="ex", body={"query": {
                                       'match_phrase' : {"content" : "ghost"}, # word/phrase to search for
                                            }
                                 })

print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print(hit["_source"]['name']) # print only the name

Got 4 Hits:
Hamlet - Shakespeare - Original.pdf
Hamlet - Shakespeare - Original.pdf
Hamlet - Shakespeare - Original.pdf
Hamlet - Shakespeare - Original.pdf
